In [3]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [4]:
data = pd.read_csv('IS_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,CTAS,2019-05-01,6892.3,1147.9,885.0,64.97,8.25,6.17,2.05,1284.4,1147.9,654.6,221.83
1,PGR,2014-12-01,19377.3,2029.1,1281.0,32.81,2.17,2.02,0.69,2029.1,2029.1,1195.1,26.99
2,ELP,2018-12-01,3857.8,583.3,363.5,14.10,1.33,1.12,0.37,676.6,583.3,306.2,7.83
3,PHM,2016-12-01,7676.5,980.2,602.7,22.59,1.76,1.82,0.36,994.0,980.2,619.1,18.38
4,F,2019-12-01,155900.0,2658.0,47.0,39.25,0.01,0.40,0.60,2658.0,2658.0,1600.5,9.30
5,WM,2011-12-01,13378.0,2081.0,961.0,28.48,2.05,1.98,1.36,2132.0,2081.0,929.5,32.71
6,SIRI,2019-12-01,7794.0,1756.0,914.0,1.73,0.20,0.19,0.05,1897.0,1756.0,851.9,7.15
7,SCCO,2017-12-01,6654.5,2618.9,728.5,8.61,0.94,1.88,0.59,2624.8,2618.9,1449.7,46.81
8,BAX,2017-12-01,10584.0,1510.0,602.0,19.49,1.11,1.59,0.61,1664.0,1510.0,865.6,64.64
9,CTSH,2014-12-01,10263.0,1885.0,1439.0,16.88,2.37,1.98,0.00,1921.0,1885.0,1201.3,52.66


In [5]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,0.013132,0.152721,0.191876,0.000414,0.000911,0.000826,0.155303,0.151308,0.152721,0.151562,221.83
1,0.036961,0.159877,0.195145,0.000208,0.000790,0.000720,0.052273,0.157298,0.159877,0.158824,26.99
2,0.007341,0.148136,0.187570,0.000089,0.000773,0.000697,0.028030,0.146419,0.148136,0.146882,7.83
3,0.014629,0.151359,0.189545,0.000143,0.000781,0.000715,0.027273,0.148972,0.151359,0.151086,18.38
4,0.297524,0.164984,0.184957,0.000249,0.000746,0.000678,0.045455,0.162357,0.164984,0.164270,9.30
5,0.025511,0.160299,0.192503,0.000181,0.000787,0.000719,0.103030,0.158126,0.160299,0.155255,32.71
6,0.014853,0.157659,0.192115,0.000010,0.000750,0.000673,0.003788,0.156236,0.157659,0.154213,7.15
7,0.012679,0.164667,0.190584,0.000054,0.000765,0.000716,0.044697,0.162090,0.164667,0.162244,46.81
8,0.020178,0.155661,0.189539,0.000123,0.000768,0.000709,0.046212,0.154362,0.155661,0.154397,64.64
9,0.019566,0.158707,0.196450,0.000107,0.000794,0.000719,0.000000,0.156429,0.158707,0.158907,52.66


In [6]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.01313249 0.15272091 0.19187598 ... 0.15130826 0.15272091 0.15156246]
 [0.03696098 0.15987705 0.19514547 ... 0.15729821 0.15987705 0.1588236 ]
 [0.00734093 0.14813585 0.18757033 ... 0.14641947 0.14813585 0.14688201]
 ...
 [0.31257509 0.34565004 0.29170268 ... 0.38301227 0.34565004 0.30593264]
 [0.00518902 0.15225639 0.19026601 ... 0.14998432 0.15225639 0.15134751]
 [0.00816982 0.15014171 0.18915389 ... 0.147815   0.15014171 0.14926791]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 5000


[221.83  26.99   7.83 ...  42.53  50.32  32.35]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 221.83
Total number of y values 5000


In [8]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=10, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [9]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [10]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 2s 409us/sample - loss: 69.6182 - mae: 69.6183 - mse: 25397.2539
Epoch 2/1000
4000/4000 [==============================] - 1s 303us/sample - loss: 54.1382 - mae: 54.1382 - mse: 22838.7754
Epoch 3/1000
4000/4000 [==============================] - 1s 248us/sample - loss: 49.1893 - mae: 49.1893 - mse: 21524.5273
Epoch 4/1000
4000/4000 [==============================] - 1s 271us/sample - loss: 48.7600 - mae: 48.7599 - mse: 21258.1914
Epoch 5/1000
4000/4000 [==============================] - 1s 265us/sample - loss: 48.8301 - mae: 48.8301 - mse: 21273.9297
Epoch 6/1000
4000/4000 [==============================] - 1s 277us/sample - loss: 48.4810 - mae: 48.4810 - mse: 21230.7383
Epoch 7/1000
4000/4000 [==============================] - 1s 273us/sample - loss: 48.6543 - mae: 48.6543 - mse: 21259.5059
Epoch 8/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 48.4359 - mae: 48.4359 - mse: 21193

4000/4000 [==============================] - 2s 388us/sample - loss: 44.7284 - mae: 44.7284 - mse: 21304.4395
Epoch 131/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 44.6062 - mae: 44.6062 - mse: 21344.5664
Epoch 132/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 44.3481 - mae: 44.3481 - mse: 21332.3789
Epoch 133/1000
4000/4000 [==============================] - 2s 411us/sample - loss: 44.3857 - mae: 44.3857 - mse: 21261.5684
Epoch 134/1000
4000/4000 [==============================] - 2s 407us/sample - loss: 44.5037 - mae: 44.5037 - mse: 21290.6348
Epoch 135/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 44.3436 - mae: 44.3436 - mse: 21257.1934
Epoch 136/1000
4000/4000 [==============================] - 1s 304us/sample - loss: 44.5741 - mae: 44.5741 - mse: 21321.2500
Epoch 137/1000
4000/4000 [==============================] - 1s 327us/sample - loss: 44.4588 - mae: 44.4588 - mse: 21286.0137
Epoch 138/1000


4000/4000 [==============================] - 1s 345us/sample - loss: 44.1690 - mae: 44.1690 - mse: 21270.7539
Epoch 261/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 43.9419 - mae: 43.9419 - mse: 21238.5254
Epoch 262/1000
4000/4000 [==============================] - 1s 349us/sample - loss: 44.5158 - mae: 44.5158 - mse: 21337.0293
Epoch 263/1000
4000/4000 [==============================] - 1s 317us/sample - loss: 44.0713 - mae: 44.0712 - mse: 21313.2852
Epoch 264/1000
4000/4000 [==============================] - 1s 338us/sample - loss: 44.2826 - mae: 44.2826 - mse: 21254.9961
Epoch 265/1000
4000/4000 [==============================] - 1s 347us/sample - loss: 44.0947 - mae: 44.0947 - mse: 21241.3613
Epoch 266/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 44.1698 - mae: 44.1698 - mse: 21232.0469
Epoch 267/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 44.1298 - mae: 44.1298 - mse: 21272.8477
Epoch 268/1000


4000/4000 [==============================] - 2s 397us/sample - loss: 43.6685 - mae: 43.6685 - mse: 21108.6113
Epoch 391/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 43.6430 - mae: 43.6430 - mse: 21113.3047
Epoch 392/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 43.8217 - mae: 43.8217 - mse: 21170.4199
Epoch 393/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 43.6492 - mae: 43.6492 - mse: 21111.9004
Epoch 394/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 43.9079 - mae: 43.9078 - mse: 21187.7148
Epoch 395/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 43.7596 - mae: 43.7596 - mse: 21109.3359
Epoch 396/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 43.6025 - mae: 43.6025 - mse: 21100.9668
Epoch 397/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 43.5476 - mae: 43.5475 - mse: 21111.5879
Epoch 398/1000


4000/4000 [==============================] - 2s 396us/sample - loss: 43.6591 - mae: 43.6591 - mse: 21049.9414
Epoch 521/1000
4000/4000 [==============================] - 2s 393us/sample - loss: 43.6114 - mae: 43.6114 - mse: 21043.6582
Epoch 522/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 43.6642 - mae: 43.6642 - mse: 21003.8496
Epoch 523/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 43.4633 - mae: 43.4633 - mse: 21013.4375
Epoch 524/1000
4000/4000 [==============================] - 2s 585us/sample - loss: 43.6127 - mae: 43.6127 - mse: 21012.8535
Epoch 525/1000
4000/4000 [==============================] - 2s 587us/sample - loss: 43.4013 - mae: 43.4013 - mse: 20971.6113
Epoch 526/1000
4000/4000 [==============================] - 2s 583us/sample - loss: 43.5695 - mae: 43.5695 - mse: 21026.4766
Epoch 527/1000
4000/4000 [==============================] - 2s 596us/sample - loss: 43.6289 - mae: 43.6289 - mse: 21037.5938
Epoch 528/1000


...

4000/4000 [==============================] - 2s 403us/sample - loss: 43.2377 - mae: 43.2377 - mse: 20733.0449
Epoch 457/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 43.1757 - mae: 43.1757 - mse: 20703.8047
Epoch 458/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 43.0586 - mae: 43.0587 - mse: 20673.8574
Epoch 459/1000
4000/4000 [==============================] - 2s 422us/sample - loss: 43.1052 - mae: 43.1052 - mse: 20704.7637
Epoch 460/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 43.0763 - mae: 43.0763 - mse: 20676.4434
Epoch 461/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 42.9967 - mae: 42.9968 - mse: 20647.6445
Epoch 462/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 43.0154 - mae: 43.0154 - mse: 20661.6680
Epoch 463/1000
4000/4000 [==============================] - 2s 431us/sample - loss: 43.0753 - mae: 43.0754 - mse: 20682.5547
Epoch 464/1000


4000/4000 [==============================] - 1s 362us/sample - loss: 43.0184 - mae: 43.0185 - mse: 20655.9727
Epoch 587/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 43.1590 - mae: 43.1590 - mse: 20694.5586
Epoch 588/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 43.0041 - mae: 43.0041 - mse: 20656.8574
Epoch 589/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 42.9485 - mae: 42.9485 - mse: 20685.6816
Epoch 590/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 42.9788 - mae: 42.9787 - mse: 20675.1504
Epoch 591/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 42.9858 - mae: 42.9858 - mse: 20695.8691
Epoch 592/1000
4000/4000 [==============================] - 1s 365us/sample - loss: 42.9363 - mae: 42.9363 - mse: 20661.9434
Epoch 593/1000
4000/4000 [==============================] - 1s 371us/sample - loss: 42.8424 - mae: 42.8424 - mse: 20647.5723
Epoch 594/1000


4000/4000 [==============================] - 2s 397us/sample - loss: 43.1343 - mae: 43.1343 - mse: 20688.7422
Epoch 717/1000
4000/4000 [==============================] - 1s 371us/sample - loss: 42.9299 - mae: 42.9299 - mse: 20657.4941
Epoch 718/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 43.2603 - mae: 43.2603 - mse: 20702.2949
Epoch 719/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 43.0895 - mae: 43.0894 - mse: 20681.6426
Epoch 720/1000
4000/4000 [==============================] - 2s 440us/sample - loss: 42.8796 - mae: 42.8796 - mse: 20645.1445
Epoch 721/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 43.0523 - mae: 43.0523 - mse: 20672.0566
Epoch 722/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 43.1443 - mae: 43.1443 - mse: 20702.7188
Epoch 723/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 42.9211 - mae: 42.9211 - mse: 20650.3125
Epoch 724/1000


4000/4000 [==============================] - 2s 384us/sample - loss: 42.9738 - mae: 42.9738 - mse: 20675.9668
Epoch 847/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 42.7027 - mae: 42.7027 - mse: 20620.3555
Epoch 848/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 43.0010 - mae: 43.0010 - mse: 20659.6367
Epoch 849/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 42.8817 - mae: 42.8817 - mse: 20643.1113
Epoch 850/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 42.9227 - mae: 42.9227 - mse: 20662.4004
Epoch 851/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 42.9707 - mae: 42.9707 - mse: 20655.3418
Epoch 852/1000
4000/4000 [==============================] - 2s 401us/sample - loss: 42.9615 - mae: 42.9615 - mse: 20686.5723
Epoch 853/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 43.0196 - mae: 43.0196 - mse: 20679.8223
Epoch 854/1000


4000/4000 [==============================] - 1s 374us/sample - loss: 42.9984 - mae: 42.9984 - mse: 20641.0000
Epoch 977/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 42.8443 - mae: 42.8443 - mse: 20663.3281
Epoch 978/1000
4000/4000 [==============================] - 1s 371us/sample - loss: 42.9332 - mae: 42.9332 - mse: 20658.8047
Epoch 979/1000
4000/4000 [==============================] - 1s 365us/sample - loss: 42.9600 - mae: 42.9600 - mse: 20619.5430
Epoch 980/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 43.0301 - mae: 43.0301 - mse: 20645.2598
Epoch 981/1000
4000/4000 [==============================] - 2s 399us/sample - loss: 43.0714 - mae: 43.0714 - mse: 20665.8301
Epoch 982/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 42.8438 - mae: 42.8438 - mse: 20636.0820
Epoch 983/1000
4000/4000 [==============================] - 1s 363us/sample - loss: 42.7321 - mae: 42.7321 - mse: 20642.6777
Epoch 984/1000


In [11]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-37.1822652  -53.64016438 -69.72896032 -40.14818001 -40.32257075]
Mean MAE: -48.20442813205719
Standard Deviation of the mean: 12.176761474356962
